<a href="https://colab.research.google.com/github/ss1111119/Indigenous-Language-e-Paradise/blob/main/%E6%96%87%E5%8C%96%E7%AF%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 文化篇
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 讀取 JSON 檔案以獲取文章編號
json_url = "https://web.klokah.tw/extension/cu_practice/textId.json"

response = requests.get(json_url)

# 定義對照表
language_groups = {
    1: "南勢阿美語",
    2: "秀姑巒阿美語",
    3: "海岸阿美語",
    4: "馬蘭阿美語",
    5: "恆春阿美語",
    6: "賽考利克泰雅語",
    7: "澤敖利泰雅語",
    8: "汶水泰雅語",
    9: "萬大泰雅語",
    10: "四季泰雅語",
    11: "宜蘭澤敖利泰雅語",
    13: "賽夏語",
    14: "邵語",
    15: "都達賽德克語",
    16: "德固達雅賽德克語",
    17: "德鹿谷賽德克語",
    18: "卓群布農語",
    19: "卡群布農語",
    20: "丹群布農語",
    21: "巒群布農語",
    22: "郡群布農語",
    23: "東排灣語",
    24: "北排灣語",
    25: "中排灣語",
    26: "南排灣語",
    27: "東魯凱語",
    28: "霧台魯凱語",
    29: "大武魯凱語",
    30: "多納魯凱語",
    31: "茂林魯凱語",
    32: "萬山魯凱語",
    33: "太魯閣語",
    34: "噶瑪蘭語",
    35: "鄒語",
    38: "南王卑南語",
    39: "知本卑南語",
    40: "西群卑南語",
    41: "建和卑南語",
    42: "雅美語",
    43: "撒奇萊雅語",
    36: "卡那卡那富語",
    37: "拉阿魯哇語"
}

# 存儲所有方言的資料
all_dialect_data = []

if response.status_code == 200:
    json_data = response.json()

    base_url = "https://web.klokah.tw/text/read_embed.php?tid={}&mode=1"

    for key_str, text_ids in json_data.items():
        key = int(key_str)  # 將鍵轉換為整數型別
        print(f"處理 JSON 編號組 {key} - {language_groups.get(key, '未知語言')}:")

        # 存儲單個方言的資料
        dialect_data = []

        for index, text_id in enumerate(text_ids, start=1):
            url = base_url.format(text_id)
            print(f"    處理第 {index} 個 JSON 編號: {text_id}")

            # 發起 GET 請求以獲取文章內容
            response = requests.get(url)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # 提取標題
                title_element = soup.find('title')
                if title_element:
                    title_text = title_element.text.strip()

                # 提取所有族語句子
                ab_sentence_elements = soup.find_all(
                    'div', class_='read-sentence Ab')
                ab_sentences = []
                for ab_element in ab_sentence_elements:
                    words = ab_element.find_all('div', class_='word')
                    sentence = ' '.join(
                        [word.text.strip() for word in words if word.text.strip() != ''])
                    ab_sentences.append(sentence)

                # 提取所有中文翻譯
                translation_elements = soup.find_all(
                    'div', class_='read-sentence Ch')
                ch_translations = []
                for translation_element in translation_elements:
                    translation = translation_element.text.strip()
                    ch_translations.append(translation)

                # 將單個文章的資料存儲為字典
                data = {'標題': title_text, '族語句子': ab_sentences,
                        '中文翻譯': ch_translations}
                dialect_data.append(data)

            else:
                print(f"    請求失敗，狀態碼: {response.status_code}")

        # 將單個方言的資料存入列表中
        all_dialect_data.append((key, dialect_data))

else:
    print(f"請求失敗，狀態碼: {response.status_code}")

# 將所有方言的資料存儲為Excel檔案
for key, dialect_data in all_dialect_data:
    df = pd.DataFrame(dialect_data)
    filename = f"{key}_{language_groups.get(key, '未知語言')}.xlsx"
    df.to_excel(filename, index=False)
    print(f"{language_groups.get(key, '未知語言')}的數據已保存至 {filename}")
